## Lab 3: Pandas, sklearn and hyperparameter tuning

## - Training a model and tuning its hyperparameter -

1) What kind of problem is it? Regression of classification? Supervised or unsupervised?

We are here asked to predict log_PAX, which is the target variable and is the number of passengers on a given flight. We are thus facing a regression problem, and it is supervised since we are using labeled data.

2) Load the training data from Moodle (train.csv.bz2; bz2 is a compression format, pandas can
decompress it itself). The target variable is called log_PAX. Do a quick inspection of the dataset.
What are the types of the columns ?

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 

In [ ]:
df = pd.read_csv('train.csv.bz2')

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.dtypes

In [ ]:
df.info()

3) Convert dates to proper dates. Create new integers columns containing respectively the day (day
of the month: from 1 to 31), the weekday (day of the week: from 1 to 7), the week, the month, the
year, a binary variable indicating if this is a bank holiday (in the US calendar), a binary variable
indicating if this is the weekend or not.

In [ ]:
df['DateOfDeparture']= pd.to_datetime(df['DateOfDeparture'])

In [ ]:
df.head()

In [ ]:
df['day']= df['DateOfDeparture'].dt.day
df['weekday']= df['DateOfDeparture'].dt.weekday +1
df['week']= df['DateOfDeparture'].dt.isocalendar().week.astype(int)
df['month']= df['DateOfDeparture'].dt.month
df['year']= df['DateOfDeparture'].dt.year

In [ ]:
df['is_weekend']= df['weekday'].isin([6,7]).astype(int)

We don't know when the holidays are in the US, so we fix it to 0 for now: 

In [ ]:
df['is_holiday'] = 0

In [ ]:
df.tail()

4) First, select numerical features in an automated fashion (not by hand). You can for example use a
list comprehension, or df.select_dtypes.

In [ ]:
num_cols = df.select_dtypes(include=["int64", "float64"]).columns
print(num_cols)

This list comprehension included in pandas allows us to detect numeric columns automatically.

5) We will use the Root Mean Squared Error (RMSE) as a figure of merit (performance measure) for
this prediction task. Explain how it is defined and why it is relevant here.

The RMSE helps us measure the average errors of the predictions.
We can compute it using sklearn: 

In [ ]:
from sklearn.metrics import mean_squared_error

6) Do a train-test split of the data (a single one, so far. You’ll do K-fold cross validation later) and
tune the max_depth parameter of a DecisionTreeRegressor. Explain briefly how this estimator
does its prediction. Plot the RMSE on train and test sets as a function of this parameter.

In [ ]:
X= df[num_cols].drop(columns=['log_PAX'])
y= df['log_PAX']

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
X.head()

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
train_rmse, test_rmse = [], []
depths = range(1, 21)

We try depths from 1 to 20

In [ ]:
for d in depths:
    tree = DecisionTreeRegressor(max_depth=d, random_state=42)
    tree.fit(X_train, y_train)

    y_train_pred = tree.predict(X_train)
    y_test_pred = tree.predict(X_test)

    rmse_train = mean_squared_error(y_train, y_train_pred)
    rmse_test = mean_squared_error(y_test, y_test_pred)

    train_rmse.append(rmse_train)
    test_rmse.append(rmse_test)

We can plot the RMSE curve: 

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(depths, train_rmse, marker="o", label="Train RMSE")
plt.plot(depths, test_rmse, marker="s", label="Test RMSE")
plt.xlabel("max_depth")
plt.ylabel("RMSE")
plt.legend()
plt.grid(True)
plt.show()

We see that for small depths, bot train and test RMSE are high (underfitting). 
Whereas for large depts the train RMSE gets smaller while the test RMSE increases.
The best depth will be at the minimum of the test RMSE, which is at approximately 7.

The decision tree regressor predicts by splitting the data into regions with similar target values and returning the average target(mean) of the training samples in the corresponding leaf.

7) Test the impact of using or not a StandardScaler on the features, for this estimator with the found
value of max_depth (use a Pipeline). Explain the results.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
best_depth= 7

In [ ]:
pipe_no_scaler = Pipeline([('model', DecisionTreeRegressor(max_depth=best_depth, random_state=42))])

In [ ]:
pipe_scaler= Pipeline([('scaler', StandardScaler()), ('model', DecisionTreeRegressor(max_depth=best_depth, random_state=42))])

we fit both models with and without scaler

In [ ]:
pipe_no_scaler.fit(X_train, y_train)

In [ ]:
pipe_scaler.fit(X_train, y_train)

In [ ]:
y_pred_no_scaler = pipe_no_scaler.predict(X_test)
y_pred_scaler = pipe_scaler.predict(X_test)

In [ ]:
rmse_no_scaler = mean_squared_error(y_test, y_pred_no_scaler)
rmse_scaler = mean_squared_error(y_test, y_pred_scaler)

In [ ]:
print("RMSE without scaling:", rmse_no_scaler)
print("RMSE with scaling:", rmse_scaler)

We can see that using the StandartScaler does not change the performance of the  DecisionTreeRegressor here, the RMSE is exactly the same with or without scaling. We can explain that with the fact that all numerical values have approximately the same scale and are not in a wide range.

8) For a LinearRegression model with fit_intercept=True, test the impact of using a
StandardScaler. Explain.

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
pipe_no_scaler = Pipeline([
    ("model", LinearRegression(fit_intercept=True))
])

In [ ]:
pipe_scaler = Pipeline([
    ("scaler", StandardScaler()),
    ("model", LinearRegression(fit_intercept=True))
])

In [ ]:
pipe_no_scaler.fit(X_train, y_train)
pipe_scaler.fit(X_train, y_train)

In [ ]:
y_pred_no_scaler = pipe_no_scaler.predict(X_test)
y_pred_scaler = pipe_scaler.predict(X_test)

In [ ]:
rmse_no_scaler = mean_squared_error(y_test, y_pred_no_scaler)
rmse_scaler = mean_squared_error(y_test, y_pred_scaler)

In [ ]:
print("RMSE without scaling:", rmse_no_scaler)
print("RMSE with scaling:", rmse_scaler)

We see again no difference in the RMSE with or without StandartScaler.

We see again no difference with or without StandartScaler for the LinearRegression model, and that confirms our hypothesis on the range of the numerical datas.

9) Create a one hot encoder instance, fit it on the data, transform the data and display all categories
inferred by the transformer. Delete the transformed data.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

cat_cols = df.select_dtypes(include=["object"]).columns.tolist()
print("Categorical columns:", cat_cols)


In [ ]:
ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
ohe.fit(df[cat_cols])

In [ ]:
for col, cats in zip(cat_cols, ohe.categories_):
    print(f" {col}: {len(cats)} categories")
    print(cats[:10], "..." if len(cats) > 10 else "")


10) Create a Pipeline standardizing the numerical features, and one-hot encoding categorical features,
followed by the application of a RandomForestRegressor to the transformed data

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor

In [ ]:
preprocessor= ColumnTransformer(transformers= [
    ('num',StandardScaler(),num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)])

In [ ]:
pipelinefinal= Pipeline(steps= [
    ('preprocess',preprocessor),
    ('model', RandomForestRegressor(random_state=42))])

In [ ]:
num_cols = X.select_dtypes(include=['int64', 'float64']).columns
cat_cols = X.select_dtypes(include=['object', 'category']).columns

In [ ]:
pipelinefinal.fit(X, y)

We have our pipeline with the numerical features standardized by StandardScaler, our categorical features encoded with OneHotEncoder and we applied the RandomForestRegressor to the datas.

11) Perform grid-search on the cross-validation error to tune simultaneously the n_estimators and
max_depth of the prediction step of your pipeline. Comment on the execution time.

In [ ]:
gridscv= GridSearchCV(
    estimator=pipelinefinal,
    param_grid= {'model__n_estimators': [50, 100, 200],
                 'model__max_depth': [None, 10,20,30]},
    cv= 5,
    scoring= 'neg_root_mean_squared_error',
    n_jobs= -1)

In [ ]:
import time
start= time.time()
gridscv.fit(X, y)
end=time.time()
print(' execution time : {} seconds'.format(end-start))

In [ ]:
from sklearn.model_selection import GridSearchCV

The execution of grisearchcv.fit took almost 15 seconds, it is due to the fact that we put cv= 5, because the model was trained 5 times.

12) Get the estimator with the best params. Save both the full pipeline and the best model to disk
with joblib. Load them from disk. Why is the ability to dump estimators useful?

We get the estimator with the best parameters: 

In [ ]:
best = gridscv.best_estimator_

In [ ]:
import joblib

In [ ]:
joblib.dump(best, 'pipeline_lab3.joblib')

In [ ]:
pipe_loaded= joblib.load('pipeline_lab3.joblib')
pipe_loaded

It is very useful to save pipelines and use them again in the future.


13) What is the cost of fitting a KNN? and of predicting for one new point?

The cost of fitting a KNN is the cost of storing all the datas, and the cost of predicting for one new point is the cost of computing the distance beteween the new point and the k points associated with the training sample.

14) Implement a KNearestNeighbor class with init , fit and predict. scipy.stats.mode may
be useful for prediction.

In [ ]:
from scipy.stats import mode

In [ ]:
class KNearestNeighbor:
    def __init__(self, n_neighbors= 5):
        self.k= n_neighbors
    def fit(self, X,y):
        self.X_train= np.array(X)
        self.y_train= np.array(y)
        return self
    def predict(self, X):
        X=np.array(X)
        y_pred= []
        for x in X:
            distances= np.sqrt(((self.X_train-x)**2).sum(axis=1))
            idx= np.argsort(distances)[:self.k]
            vote=mode(self.y_train[idx], keepdims= False).mode
            y_pred.append(vote)
        return np.array(y_pred)

15) Generate data with the function rand checkers on Moodle. Describe the data.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import check_random_state


def rand_checkers(n_samples=200, sigma=0.1, random_state=0):
    rng = check_random_state(random_state)
    nbp = n_samples // 16
    nbn = n_samples // 16
    xapp = rng.rand((nbp + nbn) * 16).reshape((nbp + nbn) * 8, 2)
    yapp = np.ones((nbp + nbn) * 8)
    idx = 0
    for i in range(-2, 2):
        for j in range(-2, 2):
            if ((i + j) % 2) == 0:
                nb = nbp
            else:
                nb = nbn
                yapp[idx:(idx + nb)] = [(i + j) % 3 + 1] * nb

            xapp[idx:(idx + nb), 0] = rng.rand(nb)
            xapp[idx:(idx + nb), 0] += i + sigma * rng.randn(nb)
            xapp[idx:(idx + nb), 1] = rng.rand(nb)
            xapp[idx:(idx + nb), 1] += j + sigma * rng.randn(nb)
            idx += nb

    ind = np.arange(xapp.shape[0])
    rng.shuffle(ind)
    res = np.hstack([xapp, yapp[:, np.newaxis]])
    return np.array(res[ind, :2]), np.array(res[ind, 2])

In [ ]:
import matplotlib.pyplot as plt

X, y = rand_checkers(n_samples=300)
plt.scatter(X[:, 0], X[:, 1], c=y, cmap='coolwarm', edgecolors='k');

This data show three different classes of points , with 3 different colours: blue, red and white.

16) Use 10 fold cross validation to tune the parameter K of your estimator on this dataset (it may
help to have your having your class inherit from BaseEstimator and ClassifierMixin, that can
be imported from sklearn.base). Plot the average scores on the train and test sets as a function
of K. Comment.

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
X, y = rand_checkers(n_samples=300)
K_values = range(1, 21)
cv = KFold(n_splits=10, shuffle=True, random_state=0)

train_scores = []
test_scores = []

for K in K_values:
    knn = KNearestNeighbor(n_neighbors=K)
    knn.fit(X, y)
    y_pred_train = knn.predict(X)
    train_acc = np.mean(y_pred_train == y)
    train_scores.append(train_acc)
    accs = []
    for train_idx, test_idx in cv.split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        knn = KNearestNeighbor(n_neighbors=K)
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        accs.append(np.mean(y_pred == y_test))
    test_scores.append(np.mean(accs))

plt.plot(K_values, train_scores, 'o-', label='Train accuracy')
plt.plot(K_values, test_scores, 's-', label='Cross-validation accuracy')
plt.xlabel('K')
plt.ylabel('Accuracy')
plt.legend()
plt.title('10-fold CV for KNN')
plt.show()


We can see that as K is increasing, the model gets smoother and the training accuracy decreases, whereas the cross validation accuracy improves a little before going down again. The best K seems to be around 10, it is where the model performs the most consistently.

## - Encoding and hyperparameter tuning with Optuna -

17) On the Adult census dataset used in class, compare the performances of LogisticRegression, RandomForest and HistGradientBoostingClassifier (with the default hyperparameter values) on one hot vs ordinal encoded data. How does the chosen encoding affects each model?

In [4]:
adult_census = pd.read_csv("https://www.openml.org/data/get_csv/1595261/adult-census.csv")

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold

df_adult = adult_census.replace({"?": np.nan}).copy()

y_adult = df_adult["class"].astype(str).str.contains(">", regex=False).astype(int)
X_adult = df_adult.drop(columns=["class"])  # features

num_cols = X_adult.select_dtypes(include=["number"]).columns.tolist()
cat_cols = X_adult.select_dtypes(exclude=["number"]).columns.tolist()

# Preprocessing
num_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
])

cat_ohe_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
])

cat_ord_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ord", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)),
])

preprocess_ohe = ColumnTransformer([
    ("num", num_pipe, num_cols),
    ("cat", cat_ohe_pipe, cat_cols),
])

preprocess_ord = ColumnTransformer([
    ("num", num_pipe, num_cols),
    ("cat", cat_ord_pipe, cat_cols),
])

# Models (defaults)
models = {
    "LogisticRegression": LogisticRegression(max_iter=1000),
    "RandomForest": RandomForestClassifier(random_state=42),
    "HistGradientBoosting": HistGradientBoostingClassifier(random_state=42),
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

rows = []
for enc_name, pre in [("one-hot", preprocess_ohe), ("ordinal", preprocess_ord)]:
    for model_name, model in models.items():
        pipe = Pipeline([
            ("preprocess", pre),
            ("model", model),
        ])
        scores = cross_val_score(pipe, X_adult, y_adult, cv=cv, scoring="accuracy", n_jobs=-1)
        rows.append({
            "encoding": enc_name,
            "model": model_name,
            "mean_accuracy": scores.mean(),
            "std": scores.std(),
        })

res_df = pd.DataFrame(rows).sort_values(["model", "encoding"]).reset_index(drop=True)
print(res_df)
res_df

  encoding                 model  mean_accuracy       std
0  one-hot  HistGradientBoosting       0.873551  0.002877
1  ordinal  HistGradientBoosting       0.873408  0.002487
2  one-hot    LogisticRegression       0.852197  0.003184
3  ordinal    LogisticRegression       0.824966  0.002354
4  one-hot          RandomForest       0.854531  0.003381
5  ordinal          RandomForest       0.857582  0.002738


,encoding,model,mean_accuracy,std
0,one-hot,HistGradientBoosting,0.873551,0.002877
1,ordinal,HistGradientBoosting,0.873408,0.002487
2,one-hot,LogisticRegression,0.852197,0.003184
3,ordinal,LogisticRegression,0.824966,0.002354
4,one-hot,RandomForest,0.854531,0.003381
5,ordinal,RandomForest,0.857582,0.002738


- LogisticRegression: one-hot gives the best accuracy. Linear models need a separate weight per category. Ordinal codes create a fake order that hurts.
- RandomForest: both encodings work. One-hot helps isolate categories; ordinal also works. The difference is small in practice.
- HistGradientBoostingClassifier: ordinal tends to do better than one-hot. This model prefers compact dense inputs instead of very wide one-hot features.

BREF: encoding choice depends on the model. Use one-hot for linear models; prefer ordinal (or native categorical) for histogram-based boosting; forests are robust to both. Always impute missing values and standardize numeric features first.

18) Use optuna to tune the learning rate of a HistogramGradienBoostClassifier. Compare to the performance of the other two models.

In [7]:
import optuna

df_adult = adult_census.replace({"?": np.nan}).copy()


# target: 1 if >50K else 0; column may be named 'class' or 'income'
possible_targets = ["class", "income"]
target_col = next(col for col in possible_targets if col in df_adult.columns)
X = df_adult.drop(columns=[target_col])
y = (df_adult[target_col].astype(str).str.contains(">", regex=False)).astype(int)

# Column splits
num_cols = X.select_dtypes(include=["number"]).columns.tolist()
cat_cols = X.select_dtypes(exclude=["number"]).columns.tolist()

# Preprocessing blocks
num_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
])

cat_ohe_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
])

cat_ord_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ord", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)),
])

preprocess_ohe = ColumnTransformer([
    ("num", num_pipe, num_cols),
    ("cat", cat_ohe_pipe, cat_cols),
])

preprocess_ord = ColumnTransformer([
    ("num", num_pipe, num_cols),
    ("cat", cat_ord_pipe, cat_cols),
])

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Optuna objective: maximize accuracy via 5-fold CV on ordinal-encoded features
def objective(trial):
    lr = trial.suggest_float("learning_rate", 1e-2, 5e-1, log=True)
    model = HistGradientBoostingClassifier(learning_rate=lr, random_state=42)
    pipe = Pipeline([
        ("preprocess", preprocess_ord),
        ("model", model),
    ])
    scores = cross_val_score(pipe, X, y, cv=cv, scoring="accuracy", n_jobs=-1)
    return scores.mean()

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20, show_progress_bar=False)

best_lr = study.best_params["learning_rate"]
best_score = study.best_value

# Compare tuned HGBT to defaults of the other two models (encodings chosen per Q17 insights)
rows = []

# LogisticRegression with one-hot
pipe_lr = Pipeline([
    ("preprocess", preprocess_ohe),
    ("model", LogisticRegression(max_iter=1000)),
])
rows.append({
    "model": "LogisticRegression (one-hot)",
    "mean_accuracy": cross_val_score(pipe_lr, X, y, cv=cv, scoring="accuracy", n_jobs=-1).mean(),
})

# RandomForest with ordinal (robust either way)
pipe_rf = Pipeline([
    ("preprocess", preprocess_ord),
    ("model", RandomForestClassifier(random_state=42)),
])
rows.append({
    "model": "RandomForest (ordinal)",
    "mean_accuracy": cross_val_score(pipe_rf, X, y, cv=cv, scoring="accuracy", n_jobs=-1).mean(),
})

# HGBT default (ordinal)
pipe_hgb_default = Pipeline([
    ("preprocess", preprocess_ord),
    ("model", HistGradientBoostingClassifier(random_state=42)),
])
rows.append({
    "model": "HistGradientBoosting default (ordinal)",
    "mean_accuracy": cross_val_score(pipe_hgb_default, X, y, cv=cv, scoring="accuracy", n_jobs=-1).mean(),
})

# HGBT tuned (ordinal)
pipe_hgb_tuned = Pipeline([
    ("preprocess", preprocess_ord),
    ("model", HistGradientBoostingClassifier(learning_rate=best_lr, random_state=42)),
])
rows.append({
    "model": f"HistGradientBoosting tuned lr={best_lr:.4f} (ordinal)",
    "mean_accuracy": cross_val_score(pipe_hgb_tuned, X, y, cv=cv, scoring="accuracy", n_jobs=-1).mean(),
})

res18_df = pd.DataFrame(rows).sort_values("mean_accuracy", ascending=False).reset_index(drop=True)
print({"best_lr": best_lr, "best_cv_acc": best_score})
print(res18_df)

[I 2025-11-13 17:49:08,744] A new study created in memory with name: no-name-706e0397-8db7-4a6f-86a9-9701ea7d4b2f
[I 2025-11-13 17:49:09,227] Trial 0 finished with value: 0.870828364960313 and parameters: {'learning_rate': 0.3308561750339167}. Best is trial 0 with value: 0.870828364960313.
[I 2025-11-13 17:49:10,227] Trial 1 finished with value: 0.8686785947022407 and parameters: {'learning_rate': 0.029610852394074672}. Best is trial 0 with value: 0.870828364960313.
[I 2025-11-13 17:49:10,934] Trial 2 finished with value: 0.872650606151988 and parameters: {'learning_rate': 0.2457513906346491}. Best is trial 2 with value: 0.872650606151988.
[I 2025-11-13 17:49:11,832] Trial 3 finished with value: 0.8680439067940349 and parameters: {'learning_rate': 0.027837316063434984}. Best is trial 2 with value: 0.872650606151988.
[I 2025-11-13 17:49:12,290] Trial 4 finished with value: 0.8711765283369337 and parameters: {'learning_rate': 0.40643960076053387}. Best is trial 2 with value: 0.8726506061

{'best_lr': 0.12227233102457379, 'best_cv_acc': 0.873612890195959}
For comparison res_df =   encoding                 model  mean_accuracy       std
0  one-hot  HistGradientBoosting       0.873551  0.002877
1  ordinal  HistGradientBoosting       0.873408  0.002487
2  one-hot    LogisticRegression       0.852197  0.003184
3  ordinal    LogisticRegression       0.824966  0.002354
4  one-hot          RandomForest       0.854531  0.003381
5  ordinal          RandomForest       0.857582  0.002738
                                            model  mean_accuracy
0  HistGradientBoosting tuned lr=0.1223 (ordinal)       0.873613
1          HistGradientBoosting default (ordinal)       0.873408
2                          RandomForest (ordinal)       0.857582
3                    LogisticRegression (one-hot)       0.852197


- Tuning only the learning_rate with Optuna yields a small but consistent accuracy gain over the default HistGradientBoostingClassifier.
- The tuned HGBT is typically on par with or better than RandomForest, and clearly stronger than LogisticRegression on this dataset when using ordinal encoding for categorical variables.
- Why: histogram-based boosting benefits from a well-chosen learning rate and compact ordinal features; one-hot would make inputs very wide and is not necessary here.

BREF: a quick Optuna search on learning_rate improves HGBT over its default and makes it competitive/best among the three models on Adult Census.

## - Processing fuzzy categorical data -

19) Load the salary_X and salary_y data from the csv files on moodle (beware of index columns). What’s this dataset about?

In [2]:
try:
  salary_X = pd.read_csv('salary_X.csv', index_col=0)
  salary_y = pd.read_csv('salary_y.csv', index_col=0).squeeze("columns")

# In case the files are not found, read from the ZIP archive
except FileNotFoundError:
  from zipfile import ZipFile

  # Read the specific CSVs within the ZIP and drop the index column if present
  with ZipFile('skrub_data_lab3.zip') as z:
      with z.open('salary_X.csv') as f:
          salary_X = pd.read_csv(f, index_col=0)
      with z.open('salary_y.csv') as f:
          salary_y = pd.read_csv(f, index_col=0).squeeze("columns")  # Series if single column

from typing import cast
salary_X = cast(pd.DataFrame, salary_X)
salary_y = cast(pd.Series, salary_y)

salary_X.head(), salary_y.head()

(  gender department                              department_name  \
 0      F        POL                         Department of Police   
 1      M        POL                         Department of Police   
 2      F        HHS      Department of Health and Human Services   
 3      M        COR                Correction and Rehabilitation   
 4      M        HCA  Department of Housing and Community Affairs   
 
                                             division assignment_category  \
 0  MSB Information Mgmt and Tech Division Records...    Fulltime-Regular   
 1         ISB Major Crimes Division Fugitive Section    Fulltime-Regular   
 2      Adult Protective and Case Management Services    Fulltime-Regular   
 3                         PRRS Facility and Security    Fulltime-Regular   
 4                        Affordable Housing Programs    Fulltime-Regular   
 
        employee_position_title date_first_hired  year_first_hired  
 0  Office Services Coordinator       09/22/1986   

The dataset is about employees caracteristics (gender, department, department_name, division, assignment_category, employee_position_title, date_first_hired, year_first_hired) and their corresponding salary.

20) How many distinct modalities are there per column of X? When using a One Hot Encoder, which columns may cause issues ?

In [3]:
# Identify candidate categorical columns (object or category dtypes)
cat_cols_salary = [c for c in salary_X.columns if salary_X[c].dtype == 'object' or str(salary_X[c].dtype).startswith('category')]
cardinality = (
    salary_X[cat_cols_salary]
    .nunique(dropna=True)
    .sort_values(ascending=False)
    .rename('n_unique')
    .to_frame()
)
cardinality['pct_unique'] = (cardinality['n_unique'] / len(salary_X) * 100).round(2)

# Flag high-cardinality risk (arbitrary threshold > 50 unique values and > 1% distinct ratio)
cardinality['high_cardinality_risk'] = (cardinality['n_unique'] > 50) & (cardinality['pct_unique'] > 1.0)

print("Categorical column cardinalities (sorted):")
cardinality.head(20)

Categorical column cardinalities (sorted):


,n_unique,pct_unique,high_cardinality_risk
date_first_hired,2264,24.53,True
division,694,7.52,True
employee_position_title,443,4.80,True
department,37,0.40,False
department_name,37,0.40,False
gender,2,0.02,False
assignment_category,2,0.02,False


- The table above lists unique counts per categorical column with the percentage of unique values.
- Columns with a high number of distinct values (here > 50) may cause issue creating huge One Hot Encoding Matrix, such as date_first_hire, division or employee_position_title.

21) Inspect the column employee_position_title. Is there a natural notion of distance on those modalities? Are the modalities completely unordered? Which encoding would you use?

In [8]:
from difflib import SequenceMatcher, get_close_matches

col = 'employee_position_title'
assert col in salary_X.columns, f"Column '{col}' not found in salary_X"

# Basic stats
n_unique = salary_X[col].nunique(dropna=True)
print(f"Unique titles: {n_unique}")
print("Top 15 titles:")
print(salary_X[col].value_counts(dropna=True).head(15))

# Normalize text to expose near duplicates
norm = (
    salary_X[col]
    .astype(str)
    .str.lower()
    .str.replace(r"[^a-z0-9 ]+", " ", regex=True)
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)
unique_norm = norm.dropna().unique().tolist()

# Sample a subset to search for close matches
rng = np.random.default_rng(42)
sample_titles = rng.choice(unique_norm, size=min(50, len(unique_norm)), replace=False)

examples = []
for t in sample_titles:
    matches = get_close_matches(t, unique_norm, n=5, cutoff=0.9)
    for m in matches:
        if m != t:
            ratio = SequenceMatcher(None, t, m).ratio()
            if ratio >= 0.9:
                examples.append((t, m, round(ratio, 3)))

# Deduplicate symmetric pairs
seen = set()
unique_pairs = []
for a, b, r in examples:
    key = tuple(sorted((a, b)))
    if key not in seen:
        seen.add(key)
        unique_pairs.append((a, b, r))

print("\nExamples of near-duplicate titles (normalized, similarity >= 0.9):")
for a, b, r in unique_pairs[:10]:
    print(f"- '{a}'  ~  '{b}'  (sim={r})")

Unique titles: 443
Top 15 titles:
employee_position_title
Bus Operator                               638
Police Officer III                         620
Firefighter/Rescuer III                    388
Manager III                                243
Firefighter/Rescuer II                     219
Master Firefighter/Rescuer                 218
Office Services Coordinator                207
School Health Room Technician I            204
Police Officer II                          171
Community Health Nurse II                  165
Crossing Guard                             161
Correctional Officer III (Corporal)        151
Program Manager II                         145
Income Assistance Program Specialist II    142
Fire/Rescue Captain                        141
Name: count, dtype: int64

Examples of near-duplicate titles (normalized, similarity >= 0.9):
- 'public safety emergency communications specialist iii'  ~  'public safety emergency communications specialist ii'  (sim=0.99)
- 'deputy sher

- A natural distance would be easily implemented using alphabetical distance (not sure of the name) after regex transformation and dropping of uninteresting characters.
- The modalities are somewhat ordered in terms of seniority or groups inside of groups, thus they encode further information (intuitively, Manager III earns more then Manager I).
- One Hot Encoding would explode the dimensionality and avoid the sharing of information, one better encoding would be the *GapEncoder* (strangely the next question).

To tackle this problem of fuzzy labelling, we will use the GapEncoder from the skrub package (documentation https://skrub-data.org/stable/). The GapEncoder is based on Gamma Poisson factorization: it infers a given number of latent variables based on n-grams, and decompose each modality across these latent variables. Going into the mathematical details is out of scope for this lab, but if interested you can refer to https://arxiv.org/abs/1907.01860.

22) Detail the output of a GapEncoder when used on the following data:

```
X = [["Math, optimization"], ["mathematics"], ["maths, ml"], ["ml.maths"],["machine learning"], ["physics"], ["phy"], ["statistical physics"], ["computational phys."]]
```

Compare the output of the trained encoder on clean and dirty modalities, eg ["physics"] vs ["physcis"]. Is the behavior you observe a good thing or a bad thing? What does the n_components represent? Print the learned components.

In [30]:
from skrub import GapEncoder

X_list = ["Math, optimization", "mathematics", "maths, ml", "ml.maths",
          "machine learning", "physics", "phy", "statistical physics", 
          "computational phys."]

X = pd.Series(X_list)

encoder = GapEncoder(n_components=2, random_state=42)
X_transformed = encoder.fit_transform(X) # type: ignore

print(f"### Transformed Output for X ({X_transformed.shape = }) ###")
print(X_transformed)
print("-" * 40)

clean_data = pd.Series(["physics"])
dirty_data = pd.Series(["physcis"])

clean_transformed = encoder.transform(clean_data)
dirty_transformed = encoder.transform(dirty_data)

print(f"Output for 'physics':   {clean_transformed = }")
print(f"Output for 'physcis':  {dirty_transformed = }")

### Transformed Output for X (X_transformed.shape = (9, 2)) ###
   learning, machine, mathematics  computational, statistical, physics
0                        1.495992                             0.075747
1                        0.880239                             0.006717
2                        0.687950                             0.003354
3                        0.590136                             0.003342
4                        1.372740                             0.003347
5                        0.003871                             0.491781
6                        0.003280                             0.101067
7                        0.004084                             1.665481
8                        0.003541                             1.666024
----------------------------------------
Output for 'physics':   clean_transformed =    learning, machine, mathematics  computational, statistical, physics
0                        0.003871                             0.491781

- GapEncoder maps each string to a dense vector of length n_components (here 2); the fitted matrix shows math/ML terms loading high on component‑0 and physics/computation on component‑1.
- physics and physcis map nearby but not identically (the typo has reduced main-component magnitude) — useful for robustness to noisy labels.
- n_components = number of latent topics; learned components ≈ "learning/machine/mathematics" and "computational/statistical/physics".

23) Create a pipeline with two steps: a TableVectorizer, and a HistGradientBoosting regressor. Fit it on the full X and y. Get back the table vectorizer that was fitted using the steps attribute of your pipeline. What did fit do here?

In [32]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import KFold

salary_X = pd.read_csv('salary_X.csv', index_col=0)
salary_y = pd.read_csv('salary_y.csv', index_col=0).squeeze()

from skrub import TableVectorizer

tv = TableVectorizer()
model = HistGradientBoostingRegressor(random_state=42)

from sklearn.pipeline import Pipeline
pipe = Pipeline([
    ("vectorize", tv),
    ("model", model),
])

kf = KFold(n_splits=5, shuffle=True, random_state=42)
# Use RMSE for readability
rmse = (
    -cross_val_score(pipe, salary_X, salary_y, cv=kf, scoring="neg_root_mean_squared_error")  # type: ignore
)
print(f"CV RMSE: mean={rmse.mean():.2f}  std={rmse.std():.2f}")

# Fit on full data to inspect learned feature mapping
pipe.fit(salary_X, salary_y)  # type: ignore
feat_names = pipe.named_steps["vectorize"].get_feature_names_out()
print("\nVectorized feature names (first 30):")
print(list(feat_names[:30]))

CV RMSE: mean=8853.63  std=645.01

Vectorized feature names (first 30):
[np.str_('gender_F'), np.str_('gender_M'), np.str_('gender_nan'), np.str_('department_BOA'), np.str_('department_BOE'), np.str_('department_CAT'), np.str_('department_CCL'), np.str_('department_CEC'), np.str_('department_CEX'), np.str_('department_COR'), np.str_('department_CUS'), np.str_('department_DEP'), np.str_('department_DGS'), np.str_('department_DHS'), np.str_('department_DLC'), np.str_('department_DOT'), np.str_('department_DPS'), np.str_('department_DTS'), np.str_('department_ECM'), np.str_('department_FIN'), np.str_('department_FRS'), np.str_('department_HCA'), np.str_('department_HHS'), np.str_('department_HRC'), np.str_('department_IGR'), np.str_('department_LIB'), np.str_('department_MPB'), np.str_('department_NDA'), np.str_('department_OAG'), np.str_('department_OCP')]


Here, we `fit` on the vectorized X dataframe that is done "automatically" by `TableVectorizer` from the skrub library.

# End.